### Things to do
- [x] Import images as separate directory
- [ ] Use os to import the images rather than hard coding it
- [x] Convertion of rgb to grayscale
- [x] Cropping the image to only manipulation zone
- [x] Resize of the image (Not required)
- [x] K-means clustering
- [ ] Extarct the major cluster points and try to change the color
- [ ] Distance Transform 

In [2]:
# Importing the image and visualizing it.

import cv2
import pandas as pd
image = cv2.imread("/home/lil-e-va/Downloads/ML_project/empty_-_detection_ML/data/image_003.jpg",0)
print(f'The size of the image: {image.shape}')

# Creating the dataframe
dataframe_image = pd.DataFrame(image)
dataframe_image.to_csv("/home/lil-e-va/Downloads/ML_project/empty_-_detection_ML/data/Image_values.csv")

cv2.imshow('Original',image)
cv2.waitKey(0)

cv2.destroyAllWindows()

The size of the image: (480, 640)


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/lil-e-va/Downloads/ML_project/ml_project/lib/python3.10/site-packages/cv2/qt/plugins"


In [3]:
'''
Cropping the image
ref - https://learnopencv.com/cropping-an-image-using-opencv/
'''

cropped_image = image[150:410,30:640]
print(cropped_image.shape)
cv2.imshow("cropped", cropped_image)

cv2.waitKey(0)
cv2.destroyAllWindows()

(260, 610)


### **Trying to change the a particular color of the pixels to other colors**

In [ ]:
'''
To pick the color values from the image the following website was used.
ref - https://imagecolorpicker.com/

Upload your image and use the eyedropper to find the color of that particular pixel.
o/p -> 150,150,150 to 185,185,185 are the free space areas from the manipulation we are going to color that.
'''

import numpy as np

img = cv2.cvtColor(cropped_image,cv2.COLOR_BGR2RGB)
vectorized = img.reshape((-1,3))
# print(f'Vectorized Image values :{vectorized}')
print(f'Vectorized Image shape :{vectorized.shape}')

# Below code is to understand the number of the rows which are accessible.

count_rows = 0
count_changed_r = 0
count_changed_g = 0
count_changed_b = 0
count_r = 0
count_g = 0
count_b = 0
count_b_w_range =0

for j in range (0,3):
    for i,value in enumerate(vectorized[:,j]):
        count_rows+=1
        # print(f'index: {i}')
        # print(f'value: {value}')
        if j==0:
            count_r+=1
            if 149<value<186:
                count_changed_r+=1    
                vectorized[i,j]=0
        elif j==1:
            count_g+=1
            if 149<value<186:
                count_changed_g+=1
                vectorized[i,j]=128
        else:
            count_b+=1
            if 149<value<186:
                count_changed_b+=1
                vectorized[i,j]=0
        if 149<value<186:
            # print(f'value satisfies the conditions are: {value}')
            count_b_w_range+=1

# print(f'rows: {count_rows}')
# print(f'Number of pixels r: {count_r}')
# print(f'Number of pixels g: {count_g}')
# print(f'Number of pixels b: {count_b}')
print(f'Number of pixels r values that changed to zero: {count_changed_r}')
print(f'Number of pixels g values that changed to zero: {count_changed_g}')
print(f'Number of pixels b values that changed to zero: {count_changed_b}')
print(f'Needs to be changed: {count_b_w_range}')

print(f'Vectorized Image values :\n{vectorized}')

vectorized = np.uint8(vectorized)
vectorized.flatten()    
result_image = vectorized.reshape((img.shape)) # It has to be reshapped to the inital image format thats when the image will be shown properly

cv2.imshow("color_change", result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()





Vectorized Image shape :(158600, 3)
Number of pixels r values that changed to zero: 84007
Number of pixels g values that changed to zero: 84007
Number of pixels b values that changed to zero: 84007
Needs to be changed: 252021
Vectorized Image values :
[[ 44  44  44]
 [ 51  51  51]
 [ 94  94  94]
 ...
 [  0 128   0]
 [  0 128   0]
 [  0 128   0]]


**Is it a wise option to resize the image??? Use Cropped Image for now.**

In [ ]:
'''
Resizing the image
ref - https://learnopencv.com/image-resizing-with-opencv/
'''

downscale_val = (256,256)
imgresized = cv2.resize(cropped_image,downscale_val,interpolation=cv2.INTER_AREA)

print(imgresized.shape)
cv2.imshow('resize_image',imgresized)
cv2.waitKey(0)

cv2.destroyAllWindows()

### _Image Segmentation with K-means_

In [ ]:
'''
Image Segmentation with Kmeans
ref - https://www.kaggle.com/code/hal1001k/image-segmentation-with-kmeans
'''

from IPython.display import Image
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
import numpy as np

In [ ]:
'''
Converting 3D image into numpy array to 2D
'''

# img = cv2.cvtColor(imgresized,cv2.COLOR_BGR2RGB)
img = cv2.cvtColor(cropped_image,cv2.COLOR_BGR2RGB)

print(f'img; {img.shape}')
r,g,b = cv2.split(img)

r = r.flatten()
g = g.flatten()
b = b.flatten()

print(f'r: {r}')
# print(f'g: {g}')
# print(f'b: {b}')

fig = plt.figure()
ax = fig.add_subplot(111,projection='3d')
# ax.scatter(r,g,b)
ax.scatter(r, g, b)
ax.set_xlabel('Red')
ax.set_ylabel('Green')
ax.set_zlabel('Blue')
plt.show()

### _Reshaping np array in 2D to perform kmeans_

In [ ]:
'''
kmeans - Clustering
ref - https://opencv24-python-tutorials.readthedocs.io/en/latest/py_tutorials/py_ml/py_kmeans/py_kmeans_opencv/py_kmeans_opencv.html#kmeans-opencv

flatten() - return a copy of a given array in such a way that it is collapsed into one dimension.

'''

vectorized = img.reshape((-1,3))
# print(f'vectorized: {vectorized.shape}')
vectorized = np.float32(vectorized)
print(f'vectorized: {vectorized}')


criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
K = 3
attempts = 10
ret,label,center=cv2.kmeans(vectorized,K,None,criteria,attempts,cv2.KMEANS_RANDOM_CENTERS)


# # Created to check the label values
# label_dict = pd.DataFrame(label)
# label_dict.to_csv('/home/lil-e-va/Downloads/ML_project/empty_-_detection_ML/data/label_data')

label = label.flatten()
print(f'{label}')



In [ ]:
# Visulaizing the labels using scatter plot

count_0 = 0
count_1 = 0
count_2 = 0
label_0 = []
label_1 = []
label_2 = []
for lab in label:
    if lab == 0:
        count_0+=1
        label_0.append(lab)

    elif lab ==1:
        count_1+=1
        label_1.append(lab)
    else:
        count_2+=1
        label_2.append(lab)

print(f'Label 0 is in nos: {count_0}')
# print(f'Label 0: {len(label_0)}')
print(f'Label 1 is in nos: {count_1}')
# print(f'Label 1: {len(label_1)}')
print(f'Label 2 is in nos: {count_2}')
# print(f'Label 2: {len(label_2)}')
print(f'center: \n{center}')

print(f'\n\n Label_0 value: {label_0}')
print(f'\n\n Label_1 value: {label_1}')
print(f'\n\n Label_2 value: {label_2}')



In [ ]:
center = np.uint8(center)
res = center[label.flatten()]
result_image = res.reshape((img.shape))

plt.imshow(result_image)
plt.show()

### _Comparing Original and Segmented Image_

In [ ]:
figure_size = 10
plt.figure(figsize=(figure_size,figure_size))
plt.subplot(1, 2, 1),plt.imshow(img)
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(1, 2, 2),plt.imshow(result_image)
plt.title('Segmented Image when K = %i' % K), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
edges = cv2.Canny(img,150,200)
plt.figure(figsize=(figure_size,figure_size))
plt.subplot(1,2,1),plt.imshow(img)
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(1,2,2),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
plt.show()

### Understand the following code to use the distance Transform

In [ ]:
image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
image = image.reshape((image.shape[0] * image.shape[1], 3))

clt = KMeans(n_clusters = 5)
clt.fit(image)

def centroid_histogram(clt):
	# grab the number of different clusters and create a histogram
	# based on the number of pixels assigned to each cluster
	numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
	(hist, _) = np.histogram(clt.labels_, bins = numLabels)
	# normalize the histogram, such that it sums to one
	hist = hist.astype("float")
	hist /= hist.sum()
	# return the histogram
	return hist

def plot_colors(hist, centroids):
	# initialize the bar chart representing the relative frequency
	# of each of the colors
	bar = np.zeros((50, 300, 3), dtype = "uint8")
	startX = 0
	# loop over the percentage of each cluster and the color of
	# each cluster
	for (percent, color) in zip(hist, centroids):
		# plot the relative percentage of each cluster
		endX = startX + (percent * 300)
		cv2.rectangle(bar, (int(startX), 0), (int(endX), 50),
			color.astype("uint8").tolist(), -1)
		startX = endX
	
	# return the bar chart
	return bar

hist = centroid_histogram(clt)
bar = plot_colors(hist, clt.cluster_centers_)
# show our color bart
plt.figure()
plt.axis("off")
plt.imshow(bar)
plt.show()

